# AI Leadership Insight & Decision Agent

**RAG-powered system for analyzing company documents and generating leadership insights.**

This notebook demonstrates the end-to-end pipeline:
1. **Ingest** company documents (PDF, DOCX, XLSX, **PPTX**, TXT) — Annual Report, Quarterly Reports, Strategy Notes, KPI Dashboard, Meeting Notes, Directives, Product Roadmap, Quarterly Review Presentation
2. **Query** the knowledge base with leadership questions
3. **Generate** structured reports with evidence, citations, and charts

**Company**: Adobe Inc.  
**Stack**: LlamaIndex · DoclingReader · MarkdownNodeParser · OpenAI gpt-4o-mini · ChromaDB · Plotly

---

## 0. Setup

In [1]:
# Install dependencies (uncomment and run on first use)
%pip install -r requirements.txt -q


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import logging
from dotenv import load_dotenv

# Load .env file if present
try:
    load_dotenv()
    print("✓ Loaded .env file")
except ImportError:
    pass  # rely on environment variable

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(name)s | %(message)s")

# Verify API key
assert os.environ.get("OPENAI_API_KEY"), (
    "Please set OPENAI_API_KEY.\n"
    "Copy .env-template → .env and add your key\n"
)
print("✓ OPENAI_API_KEY is set")

✓ Loaded .env file
✓ OPENAI_API_KEY is set


In [3]:
from src.config import load_config, initialize_llama_index

# Load configuration from config.yaml
config = load_config("config.yaml")

# Initialize LlamaIndex global settings (LLM + embedding model)
initialize_llama_index(config)

print(f"\n{'─'*55}")
print(f"  LLM       : {config.llm.model}")
print(f"  Embedding  : {config.embedding.model}")
print(f"  Retrieval  : top_k={config.retrieval.top_k}, chunk_size={config.retrieval.chunk_size}")
print(f"  ChromaDB   : collection='{config.chromadb.collection}'")
print(f"{'─'*55}")


✓ LLM: OpenAI (gpt-4o-mini)
✓ Embedding: OpenAI (text-embedding-3-small)

───────────────────────────────────────────────────────
  LLM       : gpt-4o-mini
  Embedding  : text-embedding-3-small
  Retrieval  : top_k=10, chunk_size=1024
  ChromaDB   : collection='company_docs'
───────────────────────────────────────────────────────


---
## 1. Document Ingestion

Ingest all documents from `data/sample/` through the pipeline:
- **Parse**: `DoclingReader` (LlamaIndex wrapper around IBM docling) for DOCX/XLSX/PPTX → markdown export; `pypdf` for PDF (libGL.so.1 unavailable in Codespaces); direct read for TXT
- **Chunk**: `MarkdownNodeParser` splits by markdown structure (headers, sections, tables kept atomic); `SentenceSplitter` post-pass for oversized text chunks (~1024 tokens)
- **Embed & Index**: OpenAI `text-embedding-3-small` (1536 dims) → ChromaDB (cosine similarity, MMR retrieval)

In [4]:
from src.retrieval.chromadb import list_collections, clear_collection

# Show all existing ChromaDB collections before clearing
collections = list_collections(config)
if collections:
    print(f"📦 Existing ChromaDB collections ({len(collections)}):")
    for name, count in collections:
        print(f"  • {name}: {count} vectors")
else:
    print("📦 No existing collections found")

chromadb.telemetry.product.posthog | Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


📦 Existing ChromaDB collections (1):
  • company_docs: 47 vectors


In [5]:
# Clear any previous index data (for a clean demo run)
clear_collection(config)
print("✓ ChromaDB collection cleared")

src.retrieval.chromadb | Deleted collection 'company_docs'


✓ ChromaDB collection cleared


In [6]:
from src.ingestion.pipeline import ingest_documents, filter_files

files = filter_files("data/sample")
print(f"Found {len(files)} documents:")
for f in files:
    ext = f.rsplit(".", 1)[-1].upper()
    print(f"  📄 [{ext}] {f}")

# Run the full ingestion pipeline:
#   DoclingReader → MarkdownNodeParser → OpenAI embeddings → ChromaDB
index = ingest_documents(config, data_dir="data/sample")
print(f"\n✓ Index built — ready for queries")

src.ingestion.pipeline | Ingesting 10 documents from data/sample
src.ingestion.pipeline |   → annual_report_2024.pdf
src.ingestion.pipeline |   → board_meeting_notes.txt
src.ingestion.pipeline |   → executive_directives.txt
src.ingestion.pipeline |   → kpi_dashboard.xlsx
src.ingestion.pipeline |   → monthly_metrics.xlsx
src.ingestion.pipeline |   → operational_update.docx
src.ingestion.pipeline |   → product_roadmap.pptx
src.ingestion.pipeline |   → q3_quarterly_report.pdf
src.ingestion.pipeline |   → quarterly_review_presentation.pptx
src.ingestion.pipeline |   → strategy_notes.docx
src.ingestion.parser | DoclingReader available — using AI-based document parsing
docling.datamodel.document | detected formats: [<InputFormat.PDF: 'pdf'>]
docling.document_converter | Going to convert document batch...
docling.document_converter | Initializing pipeline for StandardPdfPipeline with options hash 11ea31be9abc472bf019f6c97f75fd12


Found 10 documents:
  📄 [PDF] data/sample/annual_report_2024.pdf
  📄 [TXT] data/sample/board_meeting_notes.txt
  📄 [TXT] data/sample/executive_directives.txt
  📄 [XLSX] data/sample/kpi_dashboard.xlsx
  📄 [XLSX] data/sample/monthly_metrics.xlsx
  📄 [DOCX] data/sample/operational_update.docx
  📄 [PPTX] data/sample/product_roadmap.pptx
  📄 [PDF] data/sample/q3_quarterly_report.pdf
  📄 [PPTX] data/sample/quarterly_review_presentation.pptx
  📄 [DOCX] data/sample/strategy_notes.docx


docling.models.factories.base_factory | Loading plugin 'docling_defaults'
docling.models.factories | Registered picture descriptions: ['picture_description_vlm_engine', 'vlm', 'api']
docling.models.factories.base_factory | Loading plugin 'docling_defaults'
docling.models.factories | Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-02-21 18:37:38.681921132 [W:onnxruntime:Default, device_discovery.cc:131 GetPciBusId] Skipping pci_bus_id for PCI path at "/sys/devices/LNXSYSTM:00/LNXSYBUS:00/PNP0A03:00/device:07/VMBUS:01/5620e0c7-8062-4dce-aeb7-520c7ef76171" because filename ""5620e0c7-8062-4dce-aeb7-520c7ef76171"" dit not match expected pattern of [0-9a-f]+:[0-9a-f]+:[0-9a-f]+[.][0-9a-f]+
docling.models.stages.ocr.auto_ocr_model | rapidocr cannot be used because onnxruntime is not installed.
docling.models.stages.ocr.auto_ocr_model | easyocr cannot be used because it is not installed.
docling.models.stages.ocr.auto_ocr_model | rapidocr canno

Generating embeddings:   0%|          | 0/47 [00:00<?, ?it/s]

httpx | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
src.ingestion.pipeline | Vector index built successfully



✓ Index built — ready for queries


### 1a. Inspect Pipeline Stages (Modular)

Run individual stages of the ingestion pipeline to inspect intermediate outputs.

In [7]:
from src.ingestion.parser import parse_documents

# Stage 1: Parse — see raw document output before chunking
sample_files = filter_files("data/sample")[:3]  # inspect first 3 files
parsed_docs = parse_documents(sample_files)

print(f"📝 Parsed {len(parsed_docs)} documents from {len(sample_files)} files:\n")
for doc in parsed_docs:
    meta = doc.metadata
    preview = doc.text[:300].replace("\n", " ")
    print(f"  [{meta.get('file_type', '?')}] {meta.get('filename', '?')}  "
          f"(parser={meta.get('parser', '?')}, type={meta.get('doc_type', '?')})")
    print(f"    chars={len(doc.text):,}  |  preview: {preview}…\n")

src.ingestion.parser | DoclingReader available — using AI-based document parsing
docling.datamodel.document | detected formats: [<InputFormat.PDF: 'pdf'>]
docling.document_converter | Going to convert document batch...
docling.document_converter | Initializing pipeline for StandardPdfPipeline with options hash 11ea31be9abc472bf019f6c97f75fd12
docling.models.stages.ocr.auto_ocr_model | rapidocr cannot be used because onnxruntime is not installed.
docling.models.stages.ocr.auto_ocr_model | easyocr cannot be used because it is not installed.


docling.models.stages.ocr.auto_ocr_model | rapidocr cannot be used because rapidocr or torch is not installed.
docling.models.stages.ocr.auto_ocr_model | No OCR engine found. Please review the install details.
docling.utils.accelerator_utils | Accelerator device: 'cpu'
src.ingestion.parser | DoclingReader failed for annual_report_2024.pdf: libGL.so.1: cannot open shared object file: No such file or directory
src.ingestion.parser | Parsed: annual_report_2024.pdf (5,771 chars, parser=pypdf)
src.ingestion.parser | Parsed: board_meeting_notes.txt (5,176 chars, parser=direct)
src.ingestion.parser | Parsed: executive_directives.txt (4,153 chars, parser=direct)
src.ingestion.parser | Total documents parsed: 3


📝 Parsed 3 documents from 3 files:

  [.pdf] annual_report_2024.pdf  (parser=pypdf, type=annual)
    chars=5,771  |  preview: Adobe Inc. Annual Report 2024 Executive Summary Adobe Inc. delivered strong financial performance in fiscal year 2024, with total revenue reaching $188.5 million, representing a 23% year-over-year increase from $153.3 million in 2023. This growth was driven primarily by our Enterprise Solutions divi…

  [.txt] board_meeting_notes.txt  (parser=direct, type=meeting)
    chars=5,176  |  preview: ADOBE INC. — BOARD OF DIRECTORS MEETING MINUTES Date: October 28, 2024 Location: Adobe HQ, Building A, Conference Room 1 Attendees: Full Board (12 members), CEO, CFO, CTO, COO, CHRO, CMO  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  AGENDA ITEM 1: FY 2024 Financial Review  …

  [.txt] executive_directives.txt  (parser=direct, type=directive)
    chars=4,153  |  preview: ADOBE INC. — EXECUTIVE DIRECTIVES MEMO From: CEO Office To: All Department Head

In [8]:
from src.ingestion.chunker import chunk_documents

# Stage 2: Chunk — see how parsed documents are split into nodes
chunks = chunk_documents(parsed_docs, chunk_size=config.retrieval.chunk_size,
                         chunk_overlap=config.retrieval.chunk_overlap)

print(f"🔪 {len(chunks)} chunks from {len(parsed_docs)} documents\n")
table_chunks = [c for c in chunks if c.metadata.get("chunk_type") == "table"]
text_chunks = [c for c in chunks if c.metadata.get("chunk_type") == "text"]
print(f"  Text chunks: {len(text_chunks)}  |  Table chunks: {len(table_chunks)}\n")

# Show first 5 chunks
for i, chunk in enumerate(chunks[:5]):
    meta = chunk.metadata
    ctype = meta.get("chunk_type", "?")
    fname = meta.get("filename", "?")
    preview = chunk.text[:200].replace("\n", " ")
    print(f"  Chunk {i+1} [{ctype}] from {fname} ({len(chunk.text)} chars)")
    print(f"    → {preview}…\n")

src.ingestion.chunker | Chunking complete: 6 total (6 text, 0 table)


🔪 6 chunks from 3 documents

  Text chunks: 6  |  Table chunks: 0

  Chunk 1 [text] from annual_report_2024.pdf (3134 chars)
    → Adobe Inc. Annual Report 2024 Executive Summary Adobe Inc. delivered strong financial performance in fiscal year 2024, with total revenue reaching $188.5 million, representing a 23% year-over-year inc…

  Chunk 2 [text] from annual_report_2024.pdf (3460 chars)
    → Department Performance -- Marketing Marketing performance was mixed. While brand awareness metrics improved (website traffic up 45%, social media engagement up 67%), lead generation fell short of targ…

  Chunk 3 [text] from board_meeting_notes.txt (2968 chars)
    → ADOBE INC. — BOARD OF DIRECTORS MEETING MINUTES Date: October 28, 2024 Location: Adobe HQ, Building A, Conference Room 1 Attendees: Full Board (12 members), CEO, CFO, CTO, COO, CHRO, CMO  ━━━━━━━━━━━━…

  Chunk 4 [text] from board_meeting_notes.txt (2365 chars)
    → ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from llama_index.core import Settings

# Stage 3: Embed — see raw embedding dimensions and vector preview
sample_text = chunks[0].text[:512] if chunks else "No chunks available"
embedding = Settings.embed_model.get_text_embedding(sample_text)

print(f"🔢 Embedding model: {config.embedding.model}")
print(f"   Dimensions: {len(embedding)}")
print(f"   Sample text: {sample_text[:100]}…")
print(f"   Vector preview (first 8 dims): {[f'{v:.4f}' for v in embedding[:8]]}")

httpx | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


🔢 Embedding model: text-embedding-3-small
   Dimensions: 1536
   Sample text: Adobe Inc.
Annual Report 2024
Executive Summary
Adobe Inc. delivered strong financial performance in…
   Vector preview (first 8 dims): ['0.0372', '0.0137', '0.0727', '0.0406', '0.0275', '-0.0158', '-0.0117', '0.0392']


---
## 2. Leadership Queries

Run the three sample leadership questions through the RAG pipeline.

Each query:
1. Rewrites the question for better retrieval
2. Retrieves the top-k most relevant chunks from ChromaDB
3. Generates a structured response (summary, key points, evidence, sources)
4. Produces a chart when appropriate (trend → line, comparison → bar)

In [10]:
import os
import json
from IPython.display import display, Markdown
from src.generation.generator import query
from src.visualization.chart_builder import build_chart, format_report


def run_query(question: str) -> None:
    """Run a query and display the formatted report + chart."""
    print(f"\n{'='*60}")
    print(f"QUERY: {question}")
    print(f"{'='*60}\n")

    # Run RAG pipeline
    response = query(question, index, config)

    # Display formatted report as Markdown
    report = format_report(response)
    display(Markdown(report))

    # Display chart if available
    fig = build_chart(response)
    if fig:
        fig.show()

    # Save raw JSON response for evaluation
    os.makedirs("outputs/sample_outputs", exist_ok=True)
    filename = f"outputs/sample_outputs/{response.query_type}_{response.question[:30].replace(' ', '_')}.json"
    with open(filename, "w") as f:
        json.dump(response.model_dump(), f, indent=2)
    print(f"\n💾 Response saved to: {filename}")

    return response


### Query 1: Revenue Trend

Expected: Line chart showing quarterly revenue trajectory (Q1–Q4 2024), highlighting 23% YoY growth.

In [11]:
response_1 = run_query("What is our current revenue trend?")


QUERY: What is our current revenue trend?



httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.retrieval.query_processor | Query rewrite: 'What is our current revenue trend?' → 'What is our current revenue trend, including patterns in sales growth and overall financial performance?'
httpx | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
src.generation.generator | Retrieved 10 chunks via MMR from 6 sources (scores: ['0.405', '0.318', '0.377', '0.272', '0.319', '0.282', '0.353', '0.262', '0.360', '0.272'])
httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.generation.generator | Structured response parsed: query_type=trend
src.generation.generator | visualization is null for trend query — attempting fallback chart
httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.generation.generator | Fallback chart generated: line


📊 LEADERSHIP INSIGHT REPORT
══════════════════════════════════════════════════

Question: "What is our current revenue trend?"
Query Type: trend

📝 Summary
Adobe Inc. has demonstrated a robust revenue growth trend throughout fiscal year 2024, with quarterly revenues increasing from $42.3M in Q1 to a projected $52.4M in Q4. This reflects a consistent upward trajectory, driven primarily by the Enterprise Solutions division.

🔑 Key Points
  • Q1 2024 revenue was $42.3M, representing a 4.2% increase quarter-over-quarter (QoQ) and a 19.8% increase year-over-year (YoY).
  • Q2 2024 revenue rose to $45.1M, marking a 6.6% QoQ increase and a 21.4% YoY increase.
  • Q3 2024 revenue reached $48.7M, reflecting an 8.0% QoQ increase and a 24.2% YoY increase.
  • Q4 2024 is projected to generate $52.4M, indicating a 7.6% QoQ increase and a 26.3% YoY increase.
  • The FY 2024 total revenue is expected to be $188.5M, exceeding guidance of $187-190M.

📄 Evidence
  "Q1 2024: $42.3M (+4.2% QoQ, +19.8% YoY)"
    — annual_report_2024.pdf, §Quarterly Revenue Summary

  "Q2 2024: $45.1M (+6.6% QoQ, +21.4% YoY)"
    — annual_report_2024.pdf, §Quarterly Revenue Summary

  "Q3 2024: $48.7M (+8.0% QoQ, +24.2% YoY)"
    — quarterly_review_presentation.pptx, §Q3 2024 Financial Highlights

  "Q4 2024: $52.4M (+7.6% QoQ, +26.3% YoY)"
    — annual_report_2024.pdf, §Quarterly Revenue Summary

  "FY 2024 Total: $188.5M (+23.0% YoY)"
    — annual_report_2024.pdf, §Quarterly Revenue Summary

📋 Detailed Analysis
The revenue trend for Adobe Inc. shows a clear acceleration in growth throughout fiscal year 2024. Starting with Q1 2024 at $42.3M, the company experienced a steady increase each quarter, culminating in a projected revenue of $52.4M for Q4 2024. This consistent growth is attributed to the strong performance of the Enterprise Solutions division, which contributed significantly to the overall revenue, accounting for 63% of Q3 revenue at $30.8M. The year-over-year growth rates also reflect a positive trajectory, with Q3 showing a 24.2% increase compared to the previous year. The overall FY 2024 revenue is on track to reach $188.5M, surpassing the initial guidance of $187-190M, indicating robust demand and effective operational strategies.

📎 Sources
  1. quarterly_review_presentation.pptx → Q3 2024 Financial Highlights
  2. strategy_notes.docx
  3. annual_report_2024.pdf → Quarterly Revenue Summary
  4. q3_quarterly_report.pdf

📈 [Line Chart: Quarterly Revenue Trend for FY 2024]


💾 Response saved to: outputs/sample_outputs/trend_What_is_our_current_revenue_tr.json


### Query 2: Underperforming Departments

Expected: Bar chart comparing department scores/metrics, highlighting Marketing (72% MQL), Customer Support (68% CSAT), and HR (23% turnover).

In [12]:
response_2 = run_query("Which departments are underperforming?")


QUERY: Which departments are underperforming?



httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.retrieval.query_processor | Query rewrite: 'Which departments are underperforming?' → 'Which departments are currently underperforming or lagging in performance?'
httpx | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
src.generation.generator | Retrieved 10 chunks via MMR from 5 sources (scores: ['0.364', '0.305', '0.356', '0.311', '0.310', '0.300', '0.324', '0.332', '0.296', '0.318'])
httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.generation.generator | Structured response parsed: query_type=comparison
src.generation.generator | visualization is null for comparison query — attempting fallback chart
httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.generation.generator | Fallback chart generated: bar


📊 LEADERSHIP INSIGHT REPORT
══════════════════════════════════════════════════

Question: "Which departments are underperforming?"
Query Type: comparison

📝 Summary
The departments identified as underperforming are Marketing, Customer Support, and HR. Each of these departments has metrics significantly below target, indicating a need for immediate strategic intervention.

🔑 Key Points
  • Marketing scored 58/100, achieving only 72% of the MQL target, which is 28 percentage points below the target (quarterly_review_presentation.pptx).
  • Customer Support scored 52/100, with a CSAT of 68%, down from 82%, and an average resolution time of 7.8 hours (quarterly_review_presentation.pptx).
  • HR reported a turnover rate of 23%, significantly above the 15% target, indicating a talent retention crisis (quarterly_review_presentation.pptx).

📄 Evidence
  "Marketing: 58/100 ✗ Below Target — MQL at 72% of target, paid ROI declining."
    — quarterly_review_presentation.pptx

  "Customer Support: 52/100 ✗ Below Target — CSAT 68% (down from 82%), 7.8hr resolution."
    — quarterly_review_presentation.pptx

  "HR / People: 62/100 ✗ Below Target — 23% turnover vs. 15% target."
    — quarterly_review_presentation.pptx

  "Turnover crisis: 28% annualized, destroying institutional knowledge."
    — strategy_notes.docx

  "Employee turnover reached 23%, significantly above the 15% industry benchmark and our internal target."
    — annual_report_2024.pdf

📋 Detailed Analysis
The performance scores reveal critical issues across the underperforming departments. Marketing's score of 58/100 reflects a significant shortfall in lead generation, with MQLs at only 72% of the target, which has direct implications for the sales pipeline (quarterly_review_presentation.pptx). Customer Support's score of 52/100 is alarming, particularly with a CSAT score that has dropped to 68% from 82%, indicating declining customer satisfaction and increased resolution times (quarterly_review_presentation.pptx). The HR department's turnover rate of 23% is a major concern, as it exceeds the target of 15%, suggesting a talent retention crisis that could impact overall organizational performance (quarterly_review_presentation.pptx). This turnover is compounded by compensation issues, as noted in the strategy notes, where compensation is reported to be below market rates, further exacerbating retention challenges (strategy_notes.docx). Overall, these departments require immediate strategic intervention to address their performance deficits and align with organizational goals.

📎 Sources
  1. quarterly_review_presentation.pptx
  2. strategy_notes.docx
  3. annual_report_2024.pdf

📈 [Bar Chart: Department Performance Scores - Q3 2023]


💾 Response saved to: outputs/sample_outputs/comparison_Which_departments_are_underper.json


### Query 3: Key Risks

Expected: Detailed risk analysis with evidence from Q3 report — client concentration (35%), cybersecurity, supply chain, regulatory, talent retention.

In [13]:
response_3 = run_query("What were the key risks highlighted in the last quarter?")


QUERY: What were the key risks highlighted in the last quarter?



httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.retrieval.query_processor | Query rewrite: 'What were the key risks highlighted in the last quarter?' → 'What were the main risks and challenges identified in the last quarter?'
httpx | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
src.generation.generator | Retrieved 10 chunks via MMR from 5 sources (scores: ['0.498', '0.385', '0.338', '0.441', '0.291', '0.428', '0.300', '0.368', '0.302', '0.321'])
httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.generation.generator | Structured response parsed: query_type=risk
src.generation.generator | Extracted 1 sources from evidence: ['quarterly_review_presentation.pptx']


📊 LEADERSHIP INSIGHT REPORT
══════════════════════════════════════════════════

Question: "What were the key risks highlighted in the last quarter?"
Query Type: risk

📝 Summary
In Q3 2024, Adobe Inc. identified several critical risks impacting its operations, including client concentration, cybersecurity vulnerabilities, supply chain disruptions, regulatory compliance challenges, and high talent turnover. Each of these risks poses significant implications for revenue and operational stability.

🔑 Key Points
  • Client Concentration: Top 3 clients account for 35% of revenue ($17.0M in Q3) — a high risk for revenue stability.
  • Cybersecurity: An OAuth vulnerability was discovered on September 14, with no customer data compromised, but it highlighted weaknesses in code review processes.
  • Supply Chain: Semiconductor shortages affected 18 customers, resulting in $2.1M in deferred revenue.
  • Regulatory: The EU Digital Services Act deadline was accelerated to Q1 2025, necessitating an additional $1.2M compliance budget.
  • Talent: The annualized turnover rate reached 24.8%, with significant losses in senior engineering talent.

📄 Evidence
  "Top 3 clients = 35% of revenue ($17.0M in Q3)"
    — quarterly_review_presentation.pptx

  "OAuth vulnerability discovered Sep 14, patched within 48 hours"
    — quarterly_review_presentation.pptx

  "Semiconductor delays impacted 18 customers, $2.1M deferred revenue"
    — quarterly_review_presentation.pptx

  "EU DSA deadline accelerated to Q1 2025, $1.2M additional compliance budget"
    — quarterly_review_presentation.pptx

  "24.8% annualized turnover, Engineering lost 18 senior developers"
    — quarterly_review_presentation.pptx

📋 Detailed Analysis
The risks identified in Q3 2024 reflect a multifaceted challenge for Adobe Inc. Client concentration remains a significant concern, as the top three clients contribute 35% of total revenue, which could jeopardize financial stability if any of these clients were to reduce their business. Cybersecurity risks were underscored by the OAuth vulnerability, which, while quickly addressed, indicates potential gaps in the company's security protocols that could lead to future incidents. Supply chain issues, particularly semiconductor shortages, have already deferred $2.1M in revenue, affecting 18 customers and highlighting the need for a more resilient supply chain strategy. Regulatory pressures are increasing with the EU Digital Services Act, requiring an additional compliance budget of $1.2M, which could strain resources. Finally, talent retention is a critical issue, with a turnover rate of 24.8%, particularly in engineering, where the loss of 18 senior developers could impact product development and innovation. Addressing these risks will require strategic initiatives across client diversification, cybersecurity enhancements, supply chain management, regulatory compliance, and talent retention strategies.

📎 Sources
  1. quarterly_review_presentation.pptx



💾 Response saved to: outputs/sample_outputs/risk_What_were_the_key_risks_highli.json


---
## 3. Evaluation

Automated evaluation of response quality against the validation set.

Metrics:
- **Topic Coverage**: Does the answer mention expected key topics?
- **Factual Accuracy**: Does the answer contain expected numerical values?
- **Source Quality**: Are the expected source documents cited?
- **Chart Presence**: Is the appropriate chart type generated?

In [14]:
import re

def fuzzy_match(expected: str, text: str) -> bool:
    """Stem-aware, flexible whitespace matching.

    Handles cases like:
      - "accelerating" vs "accelerated" (stem match via 4+ char prefix)
      - "23%" appearing with different surrounding whitespace
      - Case-insensitive matching
    """
    e = expected.lower().strip()
    t = text.lower()
    # Direct substring match
    if e in t:
        return True
    # Stem-aware: if expected is 5+ chars, try matching first 4+ chars as word prefix
    if len(e) >= 5:
        stem = e[:max(4, len(e) - 3)]
        if re.search(r'\b' + re.escape(stem), t):
            return True
    # Flexible whitespace/punctuation
    pattern = re.escape(e).replace(r'\ ', r'\s+')
    if re.search(pattern, t):
        return True
    return False


# Load validation set
with open("data/evaluation/validation_set.json") as f:
    validation = json.load(f)

responses = [response_1, response_2, response_3]

print("📋 EVALUATION RESULTS")
print("=" * 60)

overall_scores = []

for vq, resp in zip(validation["questions"], responses):
    print(f"\n--- Q{vq['id']}: {vq['question']} ---")

    # Flatten ALL response text for checking (including evidence quotes)
    evidence_text = " ".join(e.quote for e in resp.answer.evidence)

    # Include visualization data (labels + values) in resp_text so chart
    # data counts toward factual accuracy — the chart IS part of the answer.
    viz_text = ""
    if resp.visualization and resp.visualization.data:
        labels = resp.visualization.data.get("labels", [])
        values = resp.visualization.data.get("values", [])
        viz_text = " ".join(str(l) for l in labels) + " " + " ".join(str(v) for v in values)

    resp_text = (
        resp.answer.summary + " "
        + " ".join(resp.answer.key_points) + " "
        + (resp.answer.detailed_analysis or "") + " "
        + evidence_text + " "
        + viz_text
    ).lower()

    # 1. Topic Coverage (fuzzy match)
    topics_found = [t for t in vq["expected_topics"] if fuzzy_match(t, resp_text)]
    topic_score = len(topics_found) / len(vq["expected_topics"]) * 100
    print(f"  Topic Coverage:   {topic_score:.0f}% ({len(topics_found)}/{len(vq['expected_topics'])})")
    print(f"    Found: {topics_found}")
    missing = [t for t in vq["expected_topics"] if not fuzzy_match(t, resp_text)]
    if missing:
        print(f"    Missing: {missing}")

    # 2. Factual Accuracy (fuzzy match)
    facts_found = [f for f in vq["expected_answer_contains"] if fuzzy_match(f, resp_text)]
    fact_score = len(facts_found) / len(vq["expected_answer_contains"]) * 100
    print(f"  Factual Accuracy: {fact_score:.0f}% ({len(facts_found)}/{len(vq['expected_answer_contains'])})")
    missing_facts = [f for f in vq["expected_answer_contains"] if not fuzzy_match(f, resp_text)]
    if missing_facts:
        print(f"    Missing: {missing_facts}")

    # 3. Source Quality
    cited_sources = [s.document_name for s in resp.sources]
    sources_found = [s for s in vq["expected_sources"] if s in cited_sources]
    source_score = len(sources_found) / len(vq["expected_sources"]) * 100
    print(f"  Source Quality:   {source_score:.0f}% ({len(sources_found)}/{len(vq['expected_sources'])})")
    print(f"    Cited: {list(set(cited_sources))}")
    missing_sources = [s for s in vq["expected_sources"] if s not in cited_sources]
    if missing_sources:
        print(f"    Missing: {missing_sources}")

    # 4. Chart Check
    expected_viz = vq["expected_visualization"]
    if expected_viz:
        has_chart = resp.visualization is not None
        correct_type = has_chart and resp.visualization.chart_type.lower() == expected_viz
        chart_status = "✅" if correct_type else ("⚠️ wrong type" if has_chart else "❌ missing")
    else:
        chart_status = "✅ (none expected)" if not resp.visualization else "⚠️ unexpected chart"
    print(f"  Chart:            {chart_status}")

    # Combined score
    combined = (topic_score + fact_score + source_score) / 3
    overall_scores.append(combined)
    print(f"  Combined Score:   {combined:.0f}%")

# Overall
avg = sum(overall_scores) / len(overall_scores)
print(f"\n{'=' * 60}")
print(f"OVERALL SCORE: {avg:.0f}%")
print(f"{'=' * 60}")

📋 EVALUATION RESULTS

--- Q1: What is our current revenue trend? ---
  Topic Coverage:   75% (3/4)
    Found: ['revenue', 'year-over-year', 'accelerating']
    Missing: ['quarterly growth']
  Factual Accuracy: 83% (5/6)
    Missing: ['23%']
  Source Quality:   100% (3/3)
    Cited: ['quarterly_review_presentation.pptx', 'strategy_notes.docx', 'q3_quarterly_report.pdf', 'annual_report_2024.pdf']
  Chart:            ✅
  Combined Score:   86%

--- Q2: Which departments are underperforming? ---
  Topic Coverage:   100% (5/5)
    Found: ['Marketing', 'Customer Support', 'HR', 'underperforming', 'below target']
  Factual Accuracy: 100% (8/8)
  Source Quality:   75% (3/4)
    Cited: ['quarterly_review_presentation.pptx', 'strategy_notes.docx', 'annual_report_2024.pdf']
    Missing: ['q3_quarterly_report.pdf']
  Chart:            ✅
  Combined Score:   92%

--- Q3: What were the key risks highlighted in the last quarter? ---
  Topic Coverage:   83% (5/6)
    Found: ['client concentration', 'cyb

---
## 4. Custom Query

Try your own leadership question below:

In [15]:
# Replace with your own question
custom_response = run_query("What is the competitive landscape and how should we respond?")


QUERY: What is the competitive landscape and how should we respond?



httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.retrieval.query_processor | Query rewrite: 'What is the competitive landscape and how should we respond?' → 'What is the current competitive environment and how should we strategically respond to market challenges and opportunities?'
httpx | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
src.generation.generator | Retrieved 10 chunks via MMR from 4 sources (scores: ['0.353', '0.285', '0.333', '0.301', '0.327', '0.298', '0.324', '0.308', '0.250', '0.280'])
httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
src.generation.generator | Structured response parsed: query_type=general


📊 LEADERSHIP INSIGHT REPORT
══════════════════════════════════════════════════

Question: "What is the competitive landscape and how should we respond?"
Query Type: general

📝 Summary
Adobe faces significant competition from emerging players like CompetitorX (InsightAI) and CompetitorY (DataPulse), both of which are targeting the analytics market with distinct strategies. To maintain our competitive edge, we must accelerate our AI Analytics Platform's general availability and enhance our deployment capabilities while leveraging our security certifications as a key differentiator.

🔑 Key Points
  • CompetitorY (DataPulse) has gained traction with 45 mid-market customers and offers a 2-week implementation time, posing a challenge to our current 6-week timeline (strategy_notes.docx).
  • CompetitorX (InsightAI) is priced 20% below our list price and has 15 enterprise customers, primarily in financial services, highlighting the need for a competitive pricing review (strategy_notes.docx).
  • Our Enterprise Solutions segment drove 62% of revenue in FY 2024, indicating strong demand for our offerings, but we must address underperformance in Marketing and Customer Support to sustain growth (board_meeting_notes.txt).

📄 Evidence
  "CompetitorY — 'DataPulse' (launched September 2024) has current traction of 45 mid-market customers and offers a 2-week implementation time."
    — strategy_notes.docx

  "CompetitorX — 'InsightAI' (launched August 2024) is priced 20% below our list price and has 15 enterprise customers."
    — strategy_notes.docx

  "Enterprise Solutions drove 62% of revenue ($116.9M), growing 30.8% YoY."
    — board_meeting_notes.txt, §AGENDA ITEM 1: FY 2024 Financial Review

📋 Detailed Analysis
The competitive landscape is evolving with the entry of CompetitorY (DataPulse) and CompetitorX (InsightAI), both of which are targeting the analytics market aggressively. DataPulse's focus on self-service analytics for the mid-market and its rapid implementation time of 2 weeks presents a direct challenge to our current offerings, which require 6 weeks for deployment. This necessitates our strategic response to accelerate the General Availability of our AI Analytics Platform and develop a Rapid Deployment Toolkit to match or exceed the implementation speed of our competitors. Additionally, InsightAI's pricing strategy, which is 20% below our list price, highlights the need for a competitive pricing review to ensure we remain attractive to potential customers. Our current financial performance indicates strong growth, with total revenue of $188.5M and a 23% YoY growth rate, but we must address the underperformance in Marketing and Customer Support to sustain this momentum. The Marketing department scored 58/100, indicating a need for a reset in strategy, particularly in Account-Based Marketing (ABM) investment, which is currently below industry standards. Furthermore, Customer Support's declining CSAT score and increased ticket resolution time pose risks to customer retention, with an estimated $28M renewal revenue at risk if CSAT drops below 65%. Addressing these internal challenges while responding to external competitive pressures will be crucial for Adobe's continued success.

📎 Sources
  1. strategy_notes.docx
  2. board_meeting_notes.txt



💾 Response saved to: outputs/sample_outputs/general_What_is_the_competitive_landsc.json
